# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [46]:
# import libraries
import pandas as pd
import numpy as np
import sys
from sqlalchemy import create_engine
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report,precision_recall_fscore_support,f1_score,accuracy_score,make_scorer, precision_score, recall_score
import pickle

nltk.download(['stopwords','punkt','wordnet'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('Disaster_categ_messages',engine)
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text=re.sub(r'[^0-9a-zA-Z]',' ',text.lower())
        
    tokens=word_tokenize(text)
    
    lemmatizer=WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tokens.append(lemmatizer.lemmatize(tok))
   
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

In [7]:
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def average_scoring_single_metric(Y_true, Y_pred,metric):
    
    target_variables=Y_true.columns

    # Calculate F1 score for each target variable
    scores = []
    for i in range(len(target_variables)):
        target_variable = target_variables[i]
        Y_true_single=Y_true[target_variables[i]]
        Y_pred_single = pd.Series(Y_pred[:,i])
        
        if metric=='f1_score':
            score = f1_score(Y_true_single, Y_pred_single,average='weighted')
        elif metric=='accuracy_score':
            score = accuracy_score(Y_true_single, Y_pred_single)
        elif metric=='recall_score':
            score = recall_score(Y_true_single, Y_pred_single,average='weighted')
        elif metric=='precision_score':
            score = precision_score(Y_true_single, Y_pred_single,average='weighted')
        elif metric=='classification_report': 
            score = classification_report(Y_true_single, Y_pred_single)
        else:
              print('Please select a metric from this list: [f1_score,accuracy_score,recall_score,precision_score,classification_report]')
              sys.exit() 
        
        scores.append(score)
        
    return scores



def average_scoring_multiple_metrics(Y_true, Y_pred):
    
    f1_average_score=np.mean(average_scoring_single_metric(Y_true, Y_pred,'f1_score'))
    accuracy_average_score=np.mean(average_scoring_single_metric(Y_true, Y_pred,'accuracy_score'))
    recall_average_score=np.mean(average_scoring_single_metric(Y_true, Y_pred,'recall_score'))
    precision_average_score=np.mean(average_scoring_single_metric(Y_true, Y_pred,'precision_score'))
    
    return (f1_average_score+accuracy_average_score+recall_average_score+precision_average_score)/4



In [9]:
Y_pred=pipeline.predict(X_test)

In [10]:
target_variables=Y_test.columns

report=average_scoring_single_metric(Y_test, Y_pred,'classification_report')

for i in range(len(target_variables)):
    target_variable = target_variables[i]
    print(f"Classification Report for {target_variable}:")
    print(report[i])
    print()

f1_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred,'f1_score'))
accuracy_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred,'accuracy_score'))
recall_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred,'recall_score'))
precision_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred,'precision_score'))
overall_average_score=average_scoring_multiple_metrics(Y_test, Y_pred)     
    
print('Average f1 score: ',f1_average_score)
print('Average accuracy score: ',accuracy_average_score)
print('Average recall score: ',recall_average_score)
print('Average precision score: ',precision_average_score)
print('Overall_average_score: ',overall_average_score)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification Report for related:
             precision    recall  f1-score   support

          0       0.66      0.37      0.47      1542
          1       0.82      0.94      0.88      4962
          2       0.55      0.42      0.48        50

avg / total       0.78      0.80      0.78      6554


Classification Report for request:
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5419
          1       0.82      0.38      0.52      1135

avg / total       0.87      0.88      0.86      6554


Classification Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554


Classification Report for aid_related:
             precision    recall  f1-score   support

          0       0.73      0.89      0.80      3810
          1       0.78      0.54      0.64      27

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average f1 score:  0.928034675305
Average accuracy score:  0.942265961415
Average recall score:  0.942265961415
Average precision score:  0.931883493123
Overall_average_score:  0.936112522814


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f7d400bad08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:

parameters = {
      'clf__estimator__bootstrap':[True,False], 
      #'clf__estimator__class_weight':[None,'balanced'],
      'clf__estimator__criterion':['gini','entropy'],
      'vect__analyzer':['word','char'],
      'vect__max_features':range(100, 301, 100),
      'tfidf__smooth_idf':[True,False] 
    }

    
custom_scorer = make_scorer(average_scoring_multiple_metrics)

cv = GridSearchCV(pipeline, param_grid=parameters,scoring=custom_scorer)

In [15]:
cv.fit(X_train,Y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__bootstrap': [True, False], 'clf__estimator__criterion': ['gini', 'entropy'], 'vect__analyzer': ['word', 'char'], 'vect__max_features': range(100, 301, 100), 'tfidf__smooth_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(average_scoring_multiple_metrics), verbose=0)

In [16]:
cv.best_params_

{'clf__estimator__bootstrap': True,
 'clf__estimator__criterion': 'entropy',
 'tfidf__smooth_idf': True,
 'vect__analyzer': 'word',
 'vect__max_features': 300}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [1]:
bootstrap=cv.best_params_['clf__estimator__bootstrap']
criterion=cv.best_params_['clf__estimator__criterion']
smooth_idf=cv.best_params_['tfidf__smooth_idf']
analyzer=cv.best_params_['vect__analyzer']
max_features=cv.best_params_['vect__max_features']




NameError: name 'cv' is not defined

In [14]:
pipeline_tuned = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize,analyzer=analyzer,max_features=max_features)),
        ('tfidf', TfidfTransformer(smooth_idf=smooth_idf)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=bootstrap,criterion=criterion)))
])

In [15]:
pipeline_tuned.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=300, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [16]:
Y_pred_tuned=pipeline_tuned.predict(X_test)

In [17]:
target_variables=Y_test.columns

report=average_scoring_single_metric(Y_test, Y_pred_tuned,'classification_report')

for i in range(len(target_variables)):
    target_variable = target_variables[i]
    print(f"Classification Report for {target_variable}:")
    print(report[i])
    print()

f1_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred_tuned,'f1_score'))
accuracy_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred_tuned,'accuracy_score'))
recall_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred_tuned,'recall_score'))
precision_average_score=np.mean(average_scoring_single_metric(Y_test, Y_pred_tuned,'precision_score'))
overall_average_score=average_scoring_multiple_metrics(Y_test, Y_pred_tuned)     


print('Average f1 score: ',f1_average_score)
print('Average accuracy score: ',accuracy_average_score)
print('Average recall score: ',recall_average_score)
print('Average precision score: ',precision_average_score)
print('Overall_average_score: ',overall_average_score)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification Report for related:
             precision    recall  f1-score   support

          0       0.60      0.42      0.49      1542
          1       0.83      0.91      0.87      4962
          2       0.44      0.36      0.40        50

avg / total       0.77      0.79      0.78      6554


Classification Report for request:
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5419
          1       0.83      0.47      0.60      1135

avg / total       0.89      0.89      0.88      6554


Classification Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554


Classification Report for aid_related:
             precision    recall  f1-score   support

          0       0.72      0.84      0.78      3810
          1       0.72      0.55      0.62      27

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average f1 score:  0.93272878057
Average accuracy score:  0.944541925203
Average recall score:  0.944541925203
Average precision score:  0.932953271689
Overall_average_score:  0.938691475666


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
pipeline_further_improved = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])

In [19]:
print(sum(Y_train.related==2))
print(sum(Y_test.related==2))

138
50


In [20]:
Y_test.loc[Y_test['related']==2, 'related'] = 1
Y_train.loc[Y_train['related']==2, 'related'] = 1



/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
print(sum(Y_train.related==2))
print(sum(Y_test.related==2))

0
0


In [34]:
for column in Y_train.columns:
    print(Y_train.groupby(column).size())

related
0     4580
1    15082
dtype: int64
request
0    16323
1     3339
dtype: int64
offer
0    19572
1       90
dtype: int64
aid_related
0    11546
1     8116
dtype: int64
medical_help
0    18106
1     1556
dtype: int64
medical_products
0    18674
1      988
dtype: int64
search_and_rescue
0    19139
1      523
dtype: int64
security
0    19318
1      344
dtype: int64
military
0    19035
1      627
dtype: int64
child_alone
0    19662
dtype: int64
water
0    18401
1     1261
dtype: int64
food
0    17482
1     2180
dtype: int64
shelter
0    17950
1     1712
dtype: int64
clothing
0    19373
1      289
dtype: int64
money
0    19208
1      454
dtype: int64
missing_people
0    19422
1      240
dtype: int64
refugees
0    19009
1      653
dtype: int64
death
0    18762
1      900
dtype: int64
other_aid
0    17085
1     2577
dtype: int64
infrastructure_related
0    18363
1     1299
dtype: int64
transport
0    18773
1      889
dtype: int64
buildings
0    18662
1     1000
dtype: int64
electricity


In [35]:
for column in Y_test.columns:
    print(Y_test.groupby(column).size())

related
0    1542
1    5012
dtype: int64
request
0    5419
1    1135
dtype: int64
offer
0    6526
1      28
dtype: int64
aid_related
0    3810
1    2744
dtype: int64
medical_help
0    6026
1     528
dtype: int64
medical_products
0    6229
1     325
dtype: int64
search_and_rescue
0    6353
1     201
dtype: int64
security
0    6427
1     127
dtype: int64
military
0    6321
1     233
dtype: int64
child_alone
0    6554
dtype: int64
water
0    6143
1     411
dtype: int64
food
0    5811
1     743
dtype: int64
shelter
0    5952
1     602
dtype: int64
clothing
0    6438
1     116
dtype: int64
money
0    6404
1     150
dtype: int64
missing_people
0    6496
1      58
dtype: int64
refugees
0    6332
1     222
dtype: int64
death
0    6260
1     294
dtype: int64
other_aid
0    5685
1     869
dtype: int64
infrastructure_related
0    6148
1     406
dtype: int64
transport
0    6242
1     312
dtype: int64
buildings
0    6221
1     333
dtype: int64
electricity
0    6437
1     117
dtype: int64
tools
0   

In [36]:
Y_train_GBC=Y_train.drop('child_alone',axis=1)

In [37]:
Y_test_GBC=Y_test.drop('child_alone',axis=1)

In [39]:
for column in Y_test_GBC.columns:
    print(Y_test_GBC.groupby(column).size())

related
0    1542
1    5012
dtype: int64
request
0    5419
1    1135
dtype: int64
offer
0    6526
1      28
dtype: int64
aid_related
0    3810
1    2744
dtype: int64
medical_help
0    6026
1     528
dtype: int64
medical_products
0    6229
1     325
dtype: int64
search_and_rescue
0    6353
1     201
dtype: int64
security
0    6427
1     127
dtype: int64
military
0    6321
1     233
dtype: int64
water
0    6143
1     411
dtype: int64
food
0    5811
1     743
dtype: int64
shelter
0    5952
1     602
dtype: int64
clothing
0    6438
1     116
dtype: int64
money
0    6404
1     150
dtype: int64
missing_people
0    6496
1      58
dtype: int64
refugees
0    6332
1     222
dtype: int64
death
0    6260
1     294
dtype: int64
other_aid
0    5685
1     869
dtype: int64
infrastructure_related
0    6148
1     406
dtype: int64
transport
0    6242
1     312
dtype: int64
buildings
0    6221
1     333
dtype: int64
electricity
0    6437
1     117
dtype: int64
tools
0    6511
1      43
dtype: int64
hospit

In [40]:
pipeline_further_improved.fit(X_train,Y_train_GBC)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1))])

In [41]:
Y_pred_further_improved=pipeline_further_improved.predict(X_test)

In [45]:
target_variables=Y_test_GBC.columns

report=average_scoring_single_metric(Y_test_GBC, Y_pred_further_improved,'classification_report')

for i in range(len(target_variables)):
    target_variable = target_variables[i]
    print(f"Classification Report for {target_variable}:")
    print(report[i])
    print()

f1_average_score=np.mean(average_scoring_single_metric(Y_test_GBC, Y_pred_further_improved,'f1_score'))
accuracy_average_score=np.mean(average_scoring_single_metric(Y_test_GBC, Y_pred_further_improved,'accuracy_score'))
recall_average_score=np.mean(average_scoring_single_metric(Y_test_GBC, Y_pred_further_improved,'recall_score'))
precision_average_score=np.mean(average_scoring_single_metric(Y_test_GBC, Y_pred_further_improved,'precision_score'))
overall_average_score=average_scoring_multiple_metrics(Y_test_GBC, Y_pred_further_improved)     


print('Average f1 score: ',f1_average_score)
print('Average accuracy score: ',accuracy_average_score)
print('Average recall score: ',recall_average_score)
print('Average precision score: ',precision_average_score)
print('Overall_average_score: ',overall_average_score)

Classification Report for related:
             precision    recall  f1-score   support

          0       0.75      0.21      0.33      1542
          1       0.80      0.98      0.88      5012

avg / total       0.79      0.80      0.75      6554


Classification Report for request:
             precision    recall  f1-score   support

          0       0.91      0.98      0.94      5419
          1       0.83      0.54      0.65      1135

avg / total       0.90      0.90      0.89      6554


Classification Report for offer:
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      6554


Classification Report for aid_related:
             precision    recall  f1-score   support

          0       0.74      0.90      0.81      3810
          1       0.80      0.56      0.66      2744

avg / total       0.77      0.76      0.75      6

### 9. Export your model as a pickle file

In [47]:
model=pipeline_further_improved

In [48]:
pickle.dump(model, open('Disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.